### Import the required libraries and inspect the data.

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [41]:
ufo_data = pd.read_csv('ufo_data.csv', on_bad_lines='skip', low_memory=False)

In [43]:
ufo_data.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

In [45]:
ufo_data

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
88674,9/9/2013 22:00,napa,ca,us,other,1200,hour,Napa UFO&#44,9/30/2013,38.2972222,-122.284444
88675,9/9/2013 22:20,vienna,va,us,circle,5,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011111,-77.265556
88676,9/9/2013 23:00,edmond,ok,us,cigar,1020,17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,9/30/2013,35.6527778,-97.477778
88677,9/9/2013 23:00,starr,sc,us,diamond,0,2 nights,On September ninth my wife and i noticed stran...,9/30/2013,34.3769444,-82.695833


### Convert data types to the correct formats and fix the ‘datetime’ column.

In [48]:
ufo_data[pd.to_datetime(ufo_data['datetime'], errors='coerce').isna()]['datetime']

166      10/10/2005 24:00
316      10/11/1994 24:00
417      10/11/2006 24:00
487      10/11/2012 24:00
567       10/1/1972 24:00
               ...       
88077      9/7/2007 24:00
88116      9/7/2010 24:00
88144      9/7/2012 24:00
88252      9/8/2003 24:00
88405      9/8/2012 24:00
Name: datetime, Length: 1220, dtype: object

In [50]:
mask = ufo_data['datetime'].str.endswith('24:00')
ufo_data.loc[mask, 'datetime'] = ufo_data.loc[mask, 'datetime'].str.replace('24:00', '00:00')
ufo_data['datetime'] = pd.to_datetime(ufo_data['datetime'], errors='coerce')
ufo_data.loc[mask, 'datetime'] += pd.Timedelta(days=1)

In [52]:
ufo_data[pd.to_datetime(ufo_data['datetime'], errors='coerce').isna()]['datetime']

Series([], Name: datetime, dtype: datetime64[ns])

In [54]:
ufo_data['duration (seconds)'] = pd.to_numeric(ufo_data['duration (seconds)'], errors='coerce')
ufo_data['date posted'] = pd.to_datetime(ufo_data['date posted'], errors='coerce')
ufo_data['latitude'] = pd.to_numeric(ufo_data['latitude'], errors='coerce')
ufo_data.dtypes

datetime                datetime64[ns]
city                            object
state                           object
country                         object
shape                           object
duration (seconds)             float64
duration (hours/min)            object
comments                        object
date posted             datetime64[ns]
latitude                       float64
longitude                      float64
dtype: object

### Check for missing values. The ‘duration (seconds)’ column contains large outliers, so for further analysis we remove zero values and keep only the 96th percentile.

In [57]:
print(ufo_data['datetime'].isna().sum())
print(ufo_data['city'].isna().sum())
print(ufo_data['state'].isna().sum())
print(ufo_data['country'].isna().sum())
print(ufo_data['shape'].isna().sum())
print(ufo_data['duration (seconds)'].isna().sum())
print(ufo_data['comments'].isna().sum())
print(ufo_data['date posted'].isna().sum())
print(ufo_data['latitude'].isna().sum())
print(ufo_data['longitude'].isna().sum())

0
0
7409
12365
2922
5
35
0
1
0


In [59]:
ufo_data = ufo_data.dropna(subset=['latitude'])

In [61]:
ufo_data['duration (seconds)'].describe()

count    8.867300e+04
mean     8.392012e+03
std      5.911601e+05
min      0.000000e+00
25%      1.500000e+01
50%      1.200000e+02
75%      6.000000e+02
max      9.783600e+07
Name: duration (seconds), dtype: float64

In [63]:
print((ufo_data['duration (seconds)'] <= 0).sum())

7027


In [65]:
ufo_data = ufo_data[ufo_data['duration (seconds)'] > 0]

In [67]:
print(ufo_data['duration (seconds)'].quantile(0.99))
print(ufo_data['duration (seconds)'].quantile(0.98))
print(ufo_data['duration (seconds)'].quantile(0.97))
print(ufo_data['duration (seconds)'].quantile(0.96))
print(ufo_data['duration (seconds)'].quantile(0.95))

14400.0
7200.0
7200.0
3607.5999999999185
3600.0


In [69]:
upper_limit = ufo_data['duration (seconds)'].quantile(0.96)
ufo_data = ufo_data[ufo_data['duration (seconds)'] <= upper_limit]

In [71]:
ufo_data['duration (seconds)'].describe()

count    78380.000000
mean       477.431587
std        770.083964
min          0.001000
25%         30.000000
50%        180.000000
75%        600.000000
max       3602.000000
Name: duration (seconds), dtype: float64

### Some country names were embedded in the ‘city’ column in parentheses instead of a separate field, so here we extract and fix this issue.

In [74]:
# Extract all content inside parentheses
ufo_data['city_info'] = ufo_data['city'].str.extract(r"\((.*?)\)")

# Remove parentheses from the city name
ufo_data['city'] = (
    ufo_data['city']
    .str.replace(r"\(.*?\)", "", regex=True)
    .str.strip()
)

# Create a mapping
mapping = {
    'de': 'Germany', 'gb': 'United Kingdom', 'us': 'United States',
    'au': 'Australia', 'ca': 'Canada',
    'canada': 'Canada', 'australia': 'Australia',
    'nsw&#44 australia': 'Australia', 'vic&#44 australia': 'Australia',
    'qld&#44 australia': 'Australia', 'south australia': 'Australia',
    'western australia': 'Australia',
    'uk/england': 'United Kingdom', 'uk/scotland': 'United Kingdom',
    'uk/wales': 'United Kingdom', 'republic of ireland': 'Ireland',
    'new zealand': 'New Zealand', 'mexico': 'Mexico', 'germany': 'Germany',
    'spain': 'Spain', 'france': 'France', 'sweden': 'Sweden',
    'italy': 'Italy', 'belgium': 'Belgium', 'norway': 'Norway',
    'denmark': 'Denmark', 'switzerland': 'Switzerland', 'argentina': 'Argentina',
    'brazil': 'Brazil', 'netherlands': 'Netherlands', 'philippines': 'Philippines',
    'south africa': 'South Africa', 'pakistan': 'Pakistan', 'japan': 'Japan',
    'china': 'China', 'israel': 'Israel', 'portugal': 'Portugal',
    'turkey': 'Turkey', 'romania': 'Romania', 'croatia': 'Croatia',
    'finland': 'Finland', 'greece': 'Greece', 'chile': 'Chile',
    'venezuela': 'Venezuela', 'costa rica': 'Costa Rica'
}

# Clean and map the extracted information from parentheses
ufo_data['city_info_clean'] = (
    ufo_data['city_info']
    .str.lower()
    .map(mapping)
)

# Normalize the country values
ufo_data['country'] = (
    ufo_data['country']
    .str.lower()
    .map(mapping)
)

# Fill missing country entries using city_info_clean
ufo_data.loc[ufo_data['country'].isna(), 'country'] = ufo_data['city_info_clean']

# Remove the temporary helper column
ufo_data.drop(columns=['city_info', 'city_info_clean'], inplace=True)


### Check missing values again. We managed to reduce the number of null entries in the country column by about one-third. After that, we prepare the data for export and then export it.

In [76]:
print(ufo_data['datetime'].isna().sum())
print(ufo_data['city'].isna().sum())
print(ufo_data['state'].isna().sum())
print(ufo_data['country'].isna().sum())
print(ufo_data['shape'].isna().sum())
print(ufo_data['duration (seconds)'].isna().sum())
print(ufo_data['comments'].isna().sum())
print(ufo_data['date posted'].isna().sum())
print(ufo_data['latitude'].isna().sum())
print(ufo_data['longitude'].isna().sum())

0
0
6333
8049
1863
0
16
0
0
0


In [78]:
cols_to_export = [
    'datetime',       
    'city',           
    'state',             
    'country',          
    'shape',             
    'duration (seconds)',
    'date posted',       
    'latitude',          
    'longitude'          
]
ufo_export = ufo_data[cols_to_export].copy()
ufo_export['city'] = ufo_export['city'].str.title()
ufo_export['state'] = ufo_export['state'].str.upper()
ufo_export['shape'] = ufo_export['shape'].str.title()


In [592]:
ufo_export.to_csv('ufo_data_clean.csv', index=False)